In [10]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
cd /content/drive/MyDrive/MSc_AI/Year 1/P5_Data_Mining

/content/drive/MyDrive/MSc_AI/Year 1/P5_Data_Mining


In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import torch

In [24]:
df = pd.read_csv("dataset_1.csv")

In [25]:
df.columns

Index(['Unnamed: 0', 'date', 'id', 'meancircumplex.arousal',
       'meancircumplex.valence', 'countappCat.builtin',
       'countappCat.communication', 'countappCat.entertainment',
       'countappCat.finance', 'countappCat.game', 'countappCat.office',
       'countappCat.other', 'countappCat.social', 'countappCat.travel',
       'countappCat.unknown', 'countappCat.utilities', 'countappCat.weather',
       'countcall', 'countcircumplex.arousal', 'countcircumplex.valence',
       'countscreen', 'countsms', 'sumactivity', 'sumappCat.builtin',
       'sumappCat.communication', 'sumappCat.entertainment',
       'sumappCat.finance', 'sumappCat.game', 'sumappCat.office',
       'sumappCat.other', 'sumappCat.social', 'sumappCat.travel',
       'sumappCat.unknown', 'sumappCat.utilities', 'sumappCat.weather',
       'sumcall', 'sumscreen', 'sumsms', 'Target',
       'time_window_sumappCat.office', 'time_window_summood',
       'time_window_countappCat.office', 'time_window_sumcircumplex.valenc

In [26]:
# A list of all unique people, we can make time series per person.
unique_people = df['id'].unique()
print(unique_people)


ts_idx = df.columns.get_loc('date')
y_idx = df.columns.get_loc('bin')

df['date'] = pd.to_datetime(df['date'])

# The amount of consequtive days:
days = 4

X = []
y = []

for person in unique_people:
    one_person_df = df[df['id'] == person]
    #print(one_person_df.columns)
    for row in one_person_df.values:
        n_days = []
        for d in range(days, 0, -1):

            prev_day = row[ts_idx] - pd.DateOffset(days=d)
            if prev_day in one_person_df['date'].values:
                day_df = one_person_df[one_person_df['date'] == prev_day]
                day_df = day_df.drop(columns=['Unnamed: 0', 'date', 'id', 'Target',
                'time_window_sumappCat.office', 'time_window_summood',
                'time_window_countappCat.office', 'time_window_sumcircumplex.valence',
                'time_window_meancircumplex.valence', 'bin', 'handmatige_bin'], axis = 1)
                arrays = day_df.values
                lst = arrays.tolist()
                n_days.append(lst[0])
            #    day_features.append(one_person_df.drop('y', axis=1))
        #n_days_tensor = torch.tensor(n_days)
        X.append(n_days)
        y.append(row[y_idx])



['AS14.07' 'AS14.13' 'AS14.05' 'AS14.02' 'AS14.19' 'AS14.33' 'AS14.12'
 'AS14.24' 'AS14.16' 'AS14.26' 'AS14.08' 'AS14.09' 'AS14.28' 'AS14.23'
 'AS14.14' 'AS14.27' 'AS14.25' 'AS14.29' 'AS14.32' 'AS14.20' 'AS14.01'
 'AS14.15' 'AS14.06' 'AS14.31' 'AS14.17' 'AS14.03' 'AS14.30']


In [31]:
X_new = []
y_new = []

for idx, i in enumerate(X):
    if len(i) ==days:
        X_new.append(i)
        new_y = [0,0,0]
        new_y[int(y[idx])] = 1
        # y_new.append(new_y)
        y_new.append(y[idx])

In [32]:
print(len(X))
print(len(X_new))
print(len(y_new))
print(X_new[0])
print(y[0])

1259
1079
1079
[[385.0, 0.3125, 0.5384615384615384, 0.1037234042553191, 0.249221183800623, 0.0166666666666666, 0.0, 0.0, 0.0, 0.0638297872340425, 0.036144578313253, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6, 0.6, 0.1554959785522788, 0.0, 0.0408269383263899, 0.1076111637251422, 0.2308672162688793, 0.1099646143193279, 0.0, 0.0, 0.0, 0.0307107264849618, 0.0651290389488635, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1981627579259837, 0.0, 4.75, 0.0, 0.4565217391304347, 0.0, 0.5, 0.5384615384615384, 0.0], [386.0, 0.1875, 0.3846153846153846, 0.3111702127659574, 0.3862928348909657, 0.0, 0.0, 0.0, 0.0, 0.1702127659574468, 0.0421686746987951, 0.0, 0.0, 0.0, 0.0, 0.0333333333333333, 0.6, 0.6, 0.3833780160857908, 0.4375, 0.2204446537959054, 0.2909640112989445, 0.2307508484976816, 0.0, 0.0, 0.0, 0.0, 0.0726249630660696, 0.045149239421335, 0.0196407775554946, 0.0106284094865941, 0.0, 0.0, 0.0333333333333333, 0.3466222324361788, 0.4375, 3.8, 0.0, 0.3478260869565217, 0.0, 0.3571428571428571, 0.3846153846153846, 0.0], [387.0, 0.2

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)


In [34]:
print(unique_people)

['AS14.07' 'AS14.13' 'AS14.05' 'AS14.02' 'AS14.19' 'AS14.33' 'AS14.12'
 'AS14.24' 'AS14.16' 'AS14.26' 'AS14.08' 'AS14.09' 'AS14.28' 'AS14.23'
 'AS14.14' 'AS14.27' 'AS14.25' 'AS14.29' 'AS14.32' 'AS14.20' 'AS14.01'
 'AS14.15' 'AS14.06' 'AS14.31' 'AS14.17' 'AS14.03' 'AS14.30']


In [35]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from imblearn.over_sampling import SMOTE


# Define a custom dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the data and target for the current index
        sample = {'data': self.data[idx], 'target': self.target[idx]}
        return sample

# Assuming you have lists of features and targets for your time series data
# Example: features = [feature_day1, feature_day2, ..., feature_day5]
# Example: targets = [target1, target2, ..., targetN]

# Convert X_train to a numpy array
X_train_array = np.array(X_train)

X_reshaped = []
for xss in X_train_array:
    new_x = []
    for xs in xss:
        for x in xs:
            new_x.append(x)
    X_reshaped.append(new_x)

print(len(X_reshaped))
print(len(y_train))

# Fit and apply the resampling
oversampler = SMOTE()
X_train_resampled_flattened, y_train_resampled = oversampler.fit_resample(X_reshaped, y_train)

# Reshape back
X_train_resampled = []
for xs in X_train_resampled_flattened:
    sub_list = [xs[i:i+len(X_train_array[0][0])] for i in range(0, len(xs), len(X_train_array[0][0]))]
    X_train_resampled.append(sub_list)


print(len(X_train_resampled))
print(len(y_train_resampled))



# Convert lists to numpy arrays
# train_features_array = np.array(X_train_resampled)
# train_targets_array = np.array(y_train_resampled)
train_features_array = np.array(X_train)
train_targets_array = np.array(y_train)
test_features_array = np.array(X_test)
test_targets_array = np.array(y_test)


# Create a dataset instance
train_dataset = TimeSeriesDataset(train_features_array, train_targets_array)
test_dataset = TimeSeriesDataset(test_features_array, test_targets_array)

# Define batch size and shuffle option
batch_size = 1
shuffle = True  # Shuffle the data for training

# Create a DataLoader instance
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)



863
863
1044
1044


In [36]:
for batch in train_loader:
    print(batch['target'])

    break

tensor([1.], dtype=torch.float64)


In [37]:
import torch
import torch.nn as nn
import tqdm
from sklearn.metrics import classification_report


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Use the output of the last time step
        return out


# Define hyperparameters
input_size = len(X_new[0][0])       # Number of features
hidden_size = len(X_new[0][0])*2     # Number of hidden units in LSTM
num_layers = 4                     # Number of LSTM layers
output_size = 3                     # Output size

# Create an instance of the LSTM model
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


# Train the model
num_epochs = 300
loss = 1
patience = 10
val_loss = 10
best_val_loss = 10
for epoch in range(num_epochs):
    for batch in train_loader:

        inputs = batch['data'].float()  # Convert inputs to float
        targets = batch['target'].long()  # Convert targets to long

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        for batch in train_loader:
            inputs = batch['data'].float()  # Convert inputs to float
            targets = batch['target'].long()
            outputs = model(inputs)
            val_loss += criterion(outputs, targets).item()
    val_loss /= len(test_loader)

    # Check for early stopping

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1

    # Print validation loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {val_loss:.4f}')

    # Check if early stopping criteria met
    if counter >= patience:
        print(f'Train loss not decreasing for {patience} epochs. Stopping training.')
        break

Epoch [1/300], Train Loss: 4.3050
Epoch [2/300], Train Loss: 4.2751
Epoch [3/300], Train Loss: 4.2676
Epoch [4/300], Train Loss: 4.2682


KeyboardInterrupt: 

In [43]:
# Set the model to evaluation mode
model.eval()

# Define a list to store the predictions
predictions = []

# Disable gradient computation
with torch.no_grad():
    # Iterate over the test DataLoader
    for batch in test_loader:
        inputs = batch['data'].float()  # Get input features for the batch
        # Forward pass
        outputs = model(inputs)
        max_values, max_indices = torch.max(outputs, dim=1)
        # Append the predictions to the list
        predictions.append(max_indices)
        print(outputs)
        print(max_indices)

# Concatenate the predictions
predictions = torch.cat(predictions, dim=0)

# Now you can evaluate the predictions as needed
y_pred = predictions.round().flatten()

print(y_pred)
print(len(y_test))

report = classification_report(y_test, y_pred)

print("Classification Report:")
print(report)

tensor([[ 0.1703,  0.2038, -0.3401]])
tensor([1])
tensor([[ 0.1700,  0.2032, -0.3393]])
tensor([1])
tensor([[ 0.1688,  0.2015, -0.3371]])
tensor([1])
tensor([[ 0.1688,  0.2015, -0.3371]])
tensor([1])
tensor([[ 0.1507,  0.1557, -0.2782]])
tensor([1])
tensor([[ 0.1697,  0.2029, -0.3389]])
tensor([1])
tensor([[ 0.1717,  0.2054, -0.3420]])
tensor([1])
tensor([[ 0.1704,  0.2039, -0.3402]])
tensor([1])
tensor([[ 0.1715,  0.2055, -0.3423]])
tensor([1])
tensor([[ 0.1688,  0.2014, -0.3370]])
tensor([1])
tensor([[ 0.1688,  0.2015, -0.3372]])
tensor([1])
tensor([[ 0.1702,  0.2035, -0.3397]])
tensor([1])
tensor([[ 0.1698,  0.2030, -0.3390]])
tensor([1])
tensor([[ 0.1708,  0.2045, -0.3410]])
tensor([1])
tensor([[ 0.1690,  0.2018, -0.3376]])
tensor([1])
tensor([[ 0.1716,  0.2056, -0.3424]])
tensor([1])
tensor([[ 0.1698,  0.2029, -0.3390]])
tensor([1])
tensor([[ 0.1695,  0.2025, -0.3384]])
tensor([1])
tensor([[ 0.1688,  0.2015, -0.3372]])
tensor([1])
tensor([[ 0.1690,  0.2017, -0.3374]])
tensor([1])


C:\Users\tomsl\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tomsl\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tomsl\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)